# Raw data length binning

The purpose of this notebook is to design a strategy to load text fragment data with perplexity ratio score feature generated by the scoring algorithm, bin it by text fragment length and get it ready for input into feature engineering. The plan is to handle data as pandas dataframes and store it using hdf5. The text fragments will be put into overlapping length bins so that feature engineering and classifier training can be conducted separately for different length regimes. This general strategy is based on early observations of classifier performance on short fragments, long fragments and un-binned fragments.

## 1. Run set-up

In [1]:
# Change working directory to parent so we can import as we would from __main__.py
print(f'Working directory: ', end = '')
%cd ..
print()

import h5py
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn.preprocessing import LabelEncoder

import configuration as config

print(f'H5py: {h5py.__version__}')
print(f'Numpy: {np.__version__}')
print(f'Pandas: {pd.__version__}')
print(f'Sklearn: {sk.__version__}')

Working directory: /mnt/arkk/llm_detector/classifier

H5py: 3.11.0
Numpy: 1.24.4
Pandas: 2.0.3
Sklearn: 1.3.2


In [2]:
# The dataset we want to bin - omit the file extension, it will be 
# added appropriately for the input and output files
dataset_name = 'falcon-7b_scores_v2_10-300_words'

# Construct input and output file paths
input_file = f'{config.HANS_DATA_PATH}/{dataset_name}.json'
output_file = f'{config.DATA_PATH}/{dataset_name}.h5'

# The bins we are planning on using
bins = {
    'combined': [0, np.inf],
    'bin_100': [1, 100],
    'bin_150': [51, 150],
    'bin_200': [101, 200],
    'bin_250': [151, 250],
    'bin_300': [201, 300],
    # 'bin_350': [251, 350],
    # 'bin_400': [301, 400],
    # 'bin_450': [351, 450],
    # 'bin_500': [401, 500],
    # 'bin_600': [451, 600]
}

## 2. Load and clean data

In [3]:
# Load the data
data_df = pd.read_json(input_file)

# Replace and remove string 'OOM' and 'NAN' values
data_df.replace('NAN', np.nan, inplace = True)
data_df.replace('OOM', np.nan, inplace = True)
data_df.dropna(inplace = True)

# Shuffle the deck, resetting the index
data_df = data_df.sample(frac = 1).reset_index(drop = True)

# Use the index to add a unique fragment id
data_df.reset_index(inplace = True)
data_df.rename({'index': 'Fragment ID'}, axis = 1, inplace = True)

# Enforce dtypes
data_df = data_df.astype({
    'Fragment ID': np.int64,
    'Source record num': np.int64,
    'Fragment length (words)': np.int64,
    'Fragment length (tokens)': np.int64,
    'Dataset': object, #pd.StringDtype(),
    'Source': object, #pd.StringDtype(),
    'Generator': object, #pd.StringDtype(),
    'String': object, #pd.StringDtype(),
    'Perplexity': np.float64,
    'Cross-perplexity': np.float64,
    'Perplexity ratio score': np.float64,
    'Reader time (seconds)': np.float64,
    'Writer time (seconds)': np.float64,
    'Reader peak memory (GB)': np.float64,
    'Writer peak memory (GB)': np.float64
})

data_df.head()

,Fragment ID,Source record num,Fragment length (words),Fragment length (tokens),Dataset,Source,Generator,String,Perplexity,Cross-perplexity,Perplexity ratio score,Reader time (seconds),Writer time (seconds),Reader peak memory (GB),Writer peak memory (GB)
0,0,977,44,57,cc_news,human,human,"just $8,900. But it damned well ought to be tr...",3.582,3.322266,1.078189,1.085517,1.166559,5.136825,5.117237
1,1,1862,211,280,cnn,human,human,They say one man's trash is another man's trea...,2.240,2.185547,1.025022,3.793024,4.196153,6.930779,6.870121
2,2,1961,184,230,cnn,synthetic,falcon7,"First World War, there was just one native of...",2.210,2.623047,0.842889,3.230670,3.604517,6.736876,6.678129
3,3,476,49,75,cc_news,synthetic,falcon7,"again in Game 5. He allowed one hit, walked tw...",2.080,2.375000,0.875822,1.874587,2.082288,5.222683,5.201081
4,4,1574,299,416,cnn,human,human,Halston Heritage Trousers at Bloomingdale's . ...,2.740,2.777344,0.986639,5.116336,5.789324,7.478645,7.436176


In [4]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17372 entries, 0 to 17371
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Fragment ID               17372 non-null  int64  
 1   Source record num         17372 non-null  int64  
 2   Fragment length (words)   17372 non-null  int64  
 3   Fragment length (tokens)  17372 non-null  int64  
 4   Dataset                   17372 non-null  object 
 5   Source                    17372 non-null  object 
 6   Generator                 17372 non-null  object 
 7   String                    17372 non-null  object 
 8   Perplexity                17372 non-null  float64
 9   Cross-perplexity          17372 non-null  float64
 10  Perplexity ratio score    17372 non-null  float64
 11  Reader time (seconds)     17372 non-null  float64
 12  Writer time (seconds)     17372 non-null  float64
 13  Reader peak memory (GB)   17372 non-null  float64
 14  Writer

## 3. Generate hdf5 data structure

OK, let's start structuring the dataset. We want two top level groups, one for training data and one for reserved testing data. Inside those groups will live datasets for the bins. We will also use attributes to save some metadata etc.

In [5]:
# Prepare the hdf5 output - create or open for read/write
output = h5py.File(output_file, 'a')

# Create the top-level groups
_ = output.require_group('training')
_ = output.require_group('testing')

print(f'Top level groups: {(list(output.keys()))}')

# Next, we need to add a group for each fragment length bin,
# and one for the un-binned data
for group in output.keys():

    # Add the un-binned data group
    _ = output.require_group(f'{group}/combined')

    # Loop on the bins and add a group for each
    for bin in bins.keys():
        _ = output.require_group(f'{group}/{bin}')

# Finally, add the bins dictionary as metadata
output.attrs.update(bins)

print(f'\nBin attributes:')
for key, value in output.attrs.items():
    print(f' {key}: {value}')

output.close()

Top level groups: ['master', 'testing', 'training']

Bin attributes:
 bin_100: [  1 100]
 bin_150: [ 51 150]
 bin_200: [101 200]
 bin_250: [151 250]
 bin_300: [201 300]
 combined: [ 0. inf]


## 4. Populate hdf5 data structure

OK, the basic data structure is ready to go. Next thing to do is add data.

In [6]:
def make_labels(training_df, testing_df):
    '''Takes training and testing dataframes, gets and encode human/synthetic
    labels, encodes labels and returns. Note: this function leaves the 
    'Source' label column in the features dataframe for downstream use in 
    feature engineering.'''

    # Get the labels
    training_labels = training_df['Source']
    testing_labels = testing_df['Source']

    # Encode string class values as integers
    label_encoder = LabelEncoder()
    label_encoder = label_encoder.fit(training_labels)
    training_labels = pd.Series(label_encoder.transform(training_labels)).astype(np.int64)
    testing_labels = pd.Series(label_encoder.transform(testing_labels)).astype(np.int64)

    return training_labels, testing_labels

In [7]:
# Reopen out hdf5 file with pandas so we can work with dataframes
data_lake = pd.HDFStore(output_file)

# Add the raw data set at the top level as 'master' incase we want it later.
data_lake['master'] = data_df

# Next, get rid of un-trainable/unnecessary features
feature_drops = [
    'Fragment ID',
    'Source record num',
    'Dataset',
    'Generator',
    'Reader time (seconds)',
    'Writer time (seconds)',
    'Reader peak memory (GB)',
    'Writer peak memory (GB)'
]

data_df.drop(feature_drops, axis = 1, inplace = True)

# Split the data into training and testing
training_df = data_df.sample(frac = 0.7, random_state = 42)
testing_df = data_df.drop(training_df.index)

# Now do the same for the bins
for bin_id, bin_range in bins.items():

    # Pull the fragments for this bin
    bin_training_df = training_df[(training_df['Fragment length (words)'] >= bin_range[0]) & (training_df['Fragment length (words)'] <= bin_range[1])]
    bin_testing_df = testing_df[(testing_df['Fragment length (words)'] >= bin_range[0]) & (testing_df['Fragment length (words)'] <= bin_range[1])]

    # Fix the index
    bin_training_df.reset_index(inplace = True, drop = True)
    bin_testing_df.reset_index(inplace = True, drop = True)

    # Split un-binned data into features and labels
    bin_training_labels, bin_testing_labels = make_labels(bin_training_df, bin_testing_df)

    # Add the data to the data lake
    data_lake.put(f'training/{bin_id}/features', bin_training_df)
    data_lake.put(f'training/{bin_id}/labels', bin_training_labels)
    data_lake.put(f'testing/{bin_id}/features', bin_training_df)
    data_lake.put(f'testing/{bin_id}/labels', bin_training_labels)

data_lake.close()

## 5. Sanity check results

In [8]:
# Open the data lake to check out the result
data_lake = h5py.File(output_file, 'a')

# Print the result
for group in data_lake.keys():
    print(f'\n{group} contains:')

    for subgroup in data_lake[group]:

        if 'bin' in subgroup:
            print(f'  {subgroup} contains: ', end = '')

            for subsubgroup in data_lake[group][subgroup].keys():
                print(f'{subsubgroup} ', end = '')
            
            print()

        else:
            print(f' {subgroup}')

data_lake.close()


master contains:
 axis0
 axis1
 block0_items
 block0_values
 block1_items
 block1_values
 block2_items
 block2_values

testing contains:
  bin_100 contains: features labels 
  bin_150 contains: features labels 
  bin_200 contains: features labels 
  bin_250 contains: features labels 
  bin_300 contains: features labels 
  combined contains: features labels 

training contains:
  bin_100 contains: features labels 
  bin_150 contains: features labels 
  bin_200 contains: features labels 
  bin_250 contains: features labels 
  bin_300 contains: features labels 
  combined contains: features labels 


In [9]:
# Reopen out hdf5 file with pandas so we can work with dataframes
data_lake = pd.HDFStore(output_file)

print('Combined training features:\n')
print(data_lake['training/combined/features'].info())
print('\nCombined training labels:\n')
print(data_lake['training/combined/labels'].info())
print('\nBin 100 training features:\n')
print(data_lake['training/bin_100/features'].info())
print('\nBin 100 training labels:\n')
print(data_lake['training/bin_100/labels'].info())

data_lake.close()

Combined training features:

<class 'pandas.core.frame.DataFrame'>
Index: 12160 entries, 0 to 12159
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Fragment length (words)   12160 non-null  int64  
 1   Fragment length (tokens)  12160 non-null  int64  
 2   Source                    12160 non-null  object 
 3   String                    12160 non-null  object 
 4   Perplexity                12160 non-null  float64
 5   Cross-perplexity          12160 non-null  float64
 6   Perplexity ratio score    12160 non-null  float64
dtypes: float64(3), int64(2), object(2)
memory usage: 760.0+ KB
None

Combined training labels:

<class 'pandas.core.series.Series'>
Index: 12160 entries, 0 to 12159
Series name: None
Non-Null Count  Dtype
--------------  -----
12160 non-null  int64
dtypes: int64(1)
memory usage: 190.0 KB
None

Bin 100 training features:

<class 'pandas.core.frame.DataFrame'>
Index: 5104

In [10]:
# Reopen out hdf5 file with pandas so we can work with dataframes
data_lake = pd.HDFStore(output_file)

for (path, subgroups, subkeys) in data_lake.walk():

    for subgroup in subgroups:

        print("GROUP: {}/{}".format(path, subgroup))

    for subkey in subkeys:

        key = "/".join([path, subkey])

        print("KEY: {}".format(key))

data_lake.close()

GROUP: /testing
GROUP: /training
KEY: /master
GROUP: /testing/bin_100
GROUP: /testing/bin_150
GROUP: /testing/bin_200
GROUP: /testing/bin_250
GROUP: /testing/bin_300
GROUP: /testing/combined
GROUP: /training/bin_100
GROUP: /training/bin_150
GROUP: /training/bin_200
GROUP: /training/bin_250
GROUP: /training/bin_300
GROUP: /training/combined
KEY: /training/bin_100/features
KEY: /training/bin_100/labels
KEY: /training/bin_150/features
KEY: /training/bin_150/labels
KEY: /training/bin_200/features
KEY: /training/bin_200/labels
KEY: /training/bin_250/features
KEY: /training/bin_250/labels
KEY: /training/bin_300/features
KEY: /training/bin_300/labels
KEY: /training/combined/features
KEY: /training/combined/labels
KEY: /testing/bin_100/features
KEY: /testing/bin_100/labels
KEY: /testing/bin_150/features
KEY: /testing/bin_150/labels
KEY: /testing/bin_200/features
KEY: /testing/bin_200/labels
KEY: /testing/bin_250/features
KEY: /testing/bin_250/labels
KEY: /testing/bin_300/features
KEY: /testing

OK, I think we are happy with this to start with. Let's move on and do some feature engineering in the bins.